# Sentiment Analysis

In [2]:
#importing data

import pandas as pd
import numpy as np

df = pd.read_csv(r'C:\Users\Administrator\Desktop\DATA\Python Test Data\movie_data.csv')

df.head(10)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
5,Leave it to Braik to put on a good show. Final...,1
6,Nathan Detroit (Frank Sinatra) is the manager ...,1
7,"To understand ""Crash Course"" in the right cont...",1
8,I've been impressed with Chavez's stance again...,1
9,This movie is directed by Renny Harlin the fin...,1


In [11]:
# At least 7 stars = Sentiment(1)
# At most 4 stars = Sentiment(0)

# Data Preprocessing

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()

example_list = np.array(df['review'][0])
print(example_list)


In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70's, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich family 

In [16]:
# removing punctuations and ther unidentifiable characters
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[|W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [17]:
#checking it out

example = df.loc[0, 'review'][-50:]
preprocessor(example)

'is seven.title (brazil): not available'

In [18]:
df['review'] = df['review'].apply(preprocessor)

# Tokenization

In [20]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [21]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop = stopwords.words('english')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


# Transforming Data into Vectors

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                       lowercase=False,
                       preprocessor=None,
                       tokenizer=tokenizer_porter,
                       use_idf=True,
                       norm='l2',
                       smooth_idf=True)    # --------> to stop division by 0

#data

X = tfidf.fit_transform(df.review)
y = df.sentiment.values

# Model

In [25]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=0.5, shuffle=False)

logreg_clf = LogisticRegressionCV(cv=5,
                                 scoring='accuracy',
                                 random_state=0,
                                 n_jobs=-1,
                                 verbose=True,
                                 max_iter=100).fit(X_train, y_train)

#saving model to disk

saved_model = open('saved_model.sav', 'wb')
pickle.dump(logreg_clf, saved_model)
saved_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  3.9min finished


# Model Evaluation

In [27]:
filename='saved_model.sav'
saved_logreg = pickle.load(open(filename, 'rb'))

evaluation = saved_logreg.score(X_test, y_test)

In [28]:
print(f'Model got an evaluation of {evaluation}')

Model got an evaluation of 0.88828
